In [1]:
import os
from os.path import join
from io import StringIO
from textwrap import dedent

import sympy
from sympy import Symbol, sqrt, cos, pi, symbols
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

import qnet
from qnet.algebra import *

import QDYN

from src.notebook_plots_v1 import plot_bs_decay, display_hamiltonian, display_eq, show_summary_dicke
from src.single_sided_network_v1 import network_slh
from src.dicke_single_model_v1 import write_dicke_single_model, err_dicke_single

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [2]:
qnet.init_printing()

In [3]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2017-06-01_mg_dicke1_QSL_traj1.json',
}
register_doit_as_IPython_magic()

In [4]:
clusterjob.JobScript.read_defaults('./config/mlhpc_cluster.ini')

$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# QSL for Single-Excitation Dicke state

## action wrappers

In [5]:
def update_config(rf, lambda_a, iter_stop):
    config = QDYN.config.read_config_file(join(rf, 'config'))
    config['oct']['iter_stop'] = iter_stop
    for pulse_config in config['pulse']:
        pulse_config['oct_lambda_a'] = lambda_a
    QDYN.config.write_config(config, join(rf, 'config'))

In [6]:
def submit_optimization(rf, n_trajs, task):
    """Asynchronously run optimization"""
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    body = dedent(r'''
    {module_load}

    cd $CLUSTERJOB_WORKDIR
    pwd
    OMP_NUM_THREADS=1 {runner} qdyn_optimize --n-trajs={n_trajs} \
         --J_T=J_T_sm .
    ''')
    taskname = "oct_%s" % task.name.split(":")[-1]
    jobscript = clusterjob.JobScript(
        body=body, jobname=taskname, workdir=rf,
        stdout='oct.log')
    runner = {
        'copper': 'aprun -B',
        'mlhpc': 'mpirun -n %d' % n_trajs}
    if jobscript.cluster_name != 'copper':
        jobscript.ppn = int(n_trajs)
        if n_trajs == 1:
            runner['mlhpc'] = ''
    jobscript.runner = runner[jobscript.cluster_name]
    jobscript.n_trajs = str(int(n_trajs))
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))

In [7]:
def wait_for_clusterjob(dumpfile):
    """Wait until the clusterjob.AsyncResult cached in the given dumpfile ends"""
    try:
        run = clusterjob.AsyncResult.load(dumpfile)
        run.wait()
        os.unlink(dumpfile)
        return run.successful()
    except OSError:
        pass

## custom uptodate routines

In [8]:
from src.qdyn_model_v1 import pulses_uptodate

## task definitions

In [9]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    root = './data/dicke1_QSL_1traj/'
    if frac_part == '00':
        rf = 'rf_%dnodes_T%s' % (row['n_nodes'], int_part)
    else:
        rf = 'rf_%dnodes_T%s_%s' % (row['n_nodes'], int_part, frac_part)
    return join(root, rf)

In [10]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    slh_models = {}
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        n_nodes = int(row['n_nodes'])
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        if n_nodes not in slh_models:
            slh_mode
        if rf in jobs:
            continue
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=True, non_herm=True,
                    lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'targets': [join(rf, 'config')],
            'uptodate': [True, ] # up to date if target exists
        }
    for job in jobs.values():
        yield job

In [11]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [12]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ], dict(n_trajs=row['n_trajs']))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [13]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

##  OCT Submission

In [14]:
params_data_str = r'''
# n_nodes   T     lambda_a  n_trajs   iter_stop
        2   1.00     0.001        1        1000
        2   1.00    0.0005        1       10000
        2   1.00    0.0005        1       15000
        2   2.00     0.001        1        1000
        2   2.00    0.0005        1       10000
        2   2.00    0.0005        1       15000
        2   3.00     0.001        1        1000
        2   3.00    0.0005        1       10000
        2   3.00    0.0005        1       15000
        2   4.00     0.001        1        1000
        2   4.00    0.0005        1       10000
        2   5.00     0.001        1        1000
        2   7.00     0.001        1        1000
        2  10.00     0.001        1        1000
        2  10.00    0.0005        1        2000
        2  20.00     0.001        1        1000
        2  50.00     0.001        1        1000
        2  50.00    0.0005        1        2000
        2  70.00     0.001        1        1000
        2  70.00    0.0005        1        2000
        3   1.00     0.001        1        1000
        3   1.00    0.0005        1       10000
        3   2.00     0.001        1        1000
        3   2.00    0.0005        1       10000
        3   3.00     0.001        1        1000
        3   3.00    0.0005        1       10000
        3   4.00     0.001        1        1000
        3   4.00    0.0005        1        4000
        3   5.00     0.001        1        1000
        3   5.00    0.0005        1        4000
        3   8.00     0.001        1        1000
        3   8.00    0.0005        1        2000
        3   9.00     0.001        1        1000
        3   9.00    0.0005        1        2000
        3  10.00     0.001        1        1000
        3  10.00    0.0005        1        2000
        3  15.00     0.001        1        1000
        3  20.00     0.001        1        1000
        3  50.00     0.001        1        1000
        3  50.00    0.0005        1        2000
        3  70.00     0.001        1        1000
        3  70.00    0.0005        1        2000
        4   0.50     0.001        1        1000
        4   1.00     0.001        1        1000
        4   1.00    0.0005        1       10000
        4   1.00    0.0005        1       15000
        4   2.00     0.001        1        1000
        4   2.00    0.0005        1       10000
        4   2.00    0.0005        1       15000
        4   3.00     0.001        1        1000
        4   3.00    0.0005        1       10000
        4   3.00    0.0005        1       15000
        4   4.00     0.001        1        1000
        4   4.00    0.0005        1       10000
        4   5.00     0.001        1        1000
        4   5.00    0.0005        1       10000
        4   6.00     0.001        1        1000
        4   6.00    0.0005        1       10000
        4   7.00     0.001        1        1000
        4   7.00    0.0005        1       10000
        4   8.00     0.001        1        1000
        4   8.00    0.0005        1        4000
        4   9.00     0.001        1        1000
        4   9.00    0.0005        1        4000
        4  10.00     0.001        1        1000
        4  10.00    0.0005        1        4000
        4  15.00     0.001        1        1000
        4  20.00     0.001        1        1000
        4  50.00     0.001        1        1000
        4  70.00     0.001        1        1000
        5   1.00     0.001        1        1000
        5   1.00    0.0005        1       10000
        5   1.00    0.0005        1       15000
        5   2.00     0.001        1        1000
        5   2.00    0.0005        1       10000
        5   2.00    0.0005        1       15000
        5   3.00     0.001        1        1000
        5   3.00    0.0005        1       10000
        5   3.00    0.0005        1       15000
        5   4.00     0.001        1        1000
        5   4.00    0.0005        1       10000
        5   4.00    0.0005        1       15000
        5   5.00     0.001        1        1000
        5   5.00    0.0005        1       10000
        5   5.00    0.0005        1       15000
        5   6.00     0.001        1        1000
        5   6.00    0.0005        1       10000
        5   7.00     0.001        1        1000
        5   7.00    0.0005        1       10000
        5   8.00     0.001        1        1000
        5   8.00    0.0005        1       10000
        5   9.00     0.001        1        1000
        5   9.00    0.0005        1       10000
        5   9.00    0.0005        1       20000
        5  10.00     0.001        1        1000
        5  10.00    0.0005        1        8000
        5  12.00     0.001        1        1000
        5  12.00    0.0005        1        2000
        5  15.00     0.001        1        1000
        5  15.00    0.0005        1        2000
        5  18.00     0.001        1        1000
        5  18.00    0.0005        1        2000
        5  20.00     0.001        1        1000
        5  20.00    0.0005        1        2000
        5  50.00     0.001        1        1000
        5  50.00    0.0005        1        2000
        5  70.00     0.001        1        1000
        5  70.00    0.0005        1        2000
        6   1.00     0.001        1        1000
        6   1.00    0.0005        1       10000
        6   2.00     0.001        1        1000
        6   2.00    0.0005        1       10000
        6   3.00     0.001        1        1000
        6   3.00    0.0005        1       10000
        6   4.00     0.001        1        1000
        6   4.00    0.0005        1       10000
        6   5.00     0.001        1        1000
        6   5.00    0.0005        1       10000
        6   5.00    0.0005        1       15000
        6   6.00    0.0005        1       10000
        6   6.00    0.0005        1       15000
        6   7.00    0.0005        1       10000
        6   7.00    0.0005        1       15000
        6   8.00     0.001        1        1000
        6   8.00    0.0005        1       10000
        6   8.00    0.0005        1       15000
        6   9.00     0.001        1        1000
        6   9.00    0.0005        1       10000
        6   9.00    0.0005        1       15000
        6  10.00     0.001        1        1000
        6  10.00    0.0005        1       10000
        6  12.00     0.001        1        1000
        6  12.00    0.0005        1       10000
        6  14.00     0.001        1        1000
        6  14.00    0.0005        1        5000
        6  14.00    0.0005        1       10000
        6  15.00     0.001        1        1000
        6  15.00    0.0005        1        2000
        6  18.00     0.001        1        1000
        6  18.00    0.0005        1        2000
        6  20.00     0.001        1        1000
        6  20.00    0.0005        1        2000
        6  50.00     0.001        1        1000
        6  50.00    0.0005        1        2000
        6  70.00     0.001        1        1000
        6  70.00    0.0005        1        2000
        7   5.00     0.001        1        1000
        7   5.00    0.0005        1       10000
        7   8.00     0.001        1        1000
        7   8.00    0.0005        1       10000
        7   9.00     0.001        1        1000
        7   9.00    0.0005        1       10000
        7  10.00     0.001        1        1000
        7  10.00    0.0005        1       10000
        7  12.00     0.001        1        1000
        7  12.00    0.0005        1       10000
        7  14.00     0.001        1        1000
        7  14.00    0.0005        1        5000
        7  14.00    0.0005        1       10000
        7  15.00     0.001        1        1000
        7  15.00    0.0005        1        2000
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['n_nodes', 'T', 'lambda_a', 'n_trajs', 'iter_stop'],
        converters={'n_nodes': int, 'T': float, 'lambda_a': float,
                    'n_trajs': int, 'iter_stop': int})

In [15]:
print(params_df.to_string())

     n_nodes     T  lambda_a  n_trajs  iter_stop
0          2   1.0    0.0010        1       1000
1          2   1.0    0.0005        1      10000
2          2   1.0    0.0005        1      15000
3          2   2.0    0.0010        1       1000
4          2   2.0    0.0005        1      10000
5          2   2.0    0.0005        1      15000
6          2   3.0    0.0010        1       1000
7          2   3.0    0.0005        1      10000
8          2   3.0    0.0005        1      15000
9          2   4.0    0.0010        1       1000
10         2   4.0    0.0005        1      10000
11         2   5.0    0.0010        1       1000
12         2   7.0    0.0010        1       1000
13         2  10.0    0.0010        1       1000
14         2  10.0    0.0005        1       2000
15         2  20.0    0.0010        1       1000
16         2  50.0    0.0010        1       1000
17         2  50.0    0.0005        1       2000
18         2  70.0    0.0010        1       1000
19         2  70.0  

In [ ]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(level=logging.DEBUG, filename='./data/dicke1_QSL_clusterjob.log')

In [ ]:
%doit -n 20 wait_for_optimization

-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T1
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T2
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T3
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T4
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T5
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T7
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T10
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T20
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T50
-- create_runfolder:./data/dicke1_QSL_1traj/rf_2nodes_T70
-- create_runfolder:./data/dicke1_QSL_1traj/rf_3nodes_T1
-- create_runfolder:./data/dicke1_QSL_1traj/rf_3nodes_T2
-- create_runfolder:./data/dicke1_QSL_1traj/rf_3nodes_T3
-- create_runfolder:./data/dicke1_QSL_1traj/rf_3nodes_T4
-- create_runfolder:./data/dicke1_QSL_1traj/rf_3nodes_T5
-- create_runfolder:./data/dicke1_QSL_1traj/rf_3nodes_T8
-- create_runfolder:./data/dicke1_QSL_1traj/rf_3nodes_T9
-- create_runfolder:./data/

-- submit_optimization:123
-- submit_optimization:142
-- submit_optimization:140
-- submit_optimization:144
-- submit_optimization:146
-- submit_optimization:154
-- submit_optimization:156
-- submit_optimization:152
-- submit_optimization:148
-- submit_optimization:150
-- submit_optimization:159
.  wait_for_optimization:102
.  wait_for_optimization:136
.  wait_for_optimization:106
.  wait_for_optimization:104
.  wait_for_optimization:129
.  wait_for_optimization:116
.  wait_for_optimization:131
.  wait_for_optimization:133
.  wait_for_optimization:126
.  wait_for_optimization:138
.  update_runfolder:4
.  update_runfolder:42
.  update_runfolder:1
.  update_runfolder:17
.  update_runfolder:10
.  update_runfolder:19
.  update_runfolder:14
.  update_runfolder:21
.  update_runfolder:29
.  update_runfolder:31
.  update_runfolder:27
.  update_runfolder:7
.  update_runfolder:35
.  update_runfolder:33
.  update_runfolder:23
.  update_runfolder:39
.  update_runfolder:25
.  update_runfolder:44
. 